In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [2]:
# load data
df=pd.read_csv('ALOI_norm.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att19,att20,att21,att22,att23,att24,att25,att26,att27,outlier
0,0.784999,0.000018,0.0,0.000093,0.0,0.0,0.0,0.0,0.0,0.139811,...,0.048171,0.001189,0.0,0.115728,0.023449,0.0002,0.000220,0.034952,0.046914,1
1,0.958088,0.000000,0.0,0.001671,0.0,0.0,0.0,0.0,0.0,0.019556,...,0.000000,0.000000,0.0,0.022035,0.007516,0.0000,0.000000,0.001033,0.000000,1
2,0.938768,0.000000,0.0,0.005146,0.0,0.0,0.0,0.0,0.0,0.018451,...,0.000000,0.000000,0.0,0.035542,0.011982,0.0000,0.001595,0.019520,0.000000,1
3,0.954775,0.000000,0.0,0.001427,0.0,0.0,0.0,0.0,0.0,0.024944,...,0.000000,0.000000,0.0,0.019941,0.000805,0.0000,0.000000,0.000035,0.000000,1
4,0.933601,0.000000,0.0,0.001682,0.0,0.0,0.0,0.0,0.0,0.037002,...,0.000000,0.000000,0.0,0.046759,0.002663,0.0000,0.000339,0.001359,0.000000,1


In [3]:
df.shape

(50000, 28)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/aloi.json")
gc = GCForest(config)

In [9]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,27]
X = df[:,0:27]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
# number of class
len(np.unique(y_train))

2

In [10]:
         # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-05-15 00:39:20,968][cascade_classifier.fit_transform] X_groups_train.shape=[(33500, 27)],y_train.shape=(33500,),X_groups_test.shape=[(16500, 27)],y_test.shape=(16500,)
[ 2018-05-15 00:39:20,973][cascade_classifier.fit_transform] group_dims=[27]
[ 2018-05-15 00:39:20,974][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 00:39:20,976][cascade_classifier.fit_transform] group_ends=[27]
[ 2018-05-15 00:39:20,977][cascade_classifier.fit_transform] X_train.shape=(33500, 27),X_test.shape=(16500, 27)
[ 2018-05-15 00:39:20,982][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(33500, 27), X_cur_test.shape=(16500, 27)
[ 2018-05-15 00:39:25,373][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.49%
[ 2018-05-15 00:39:30,517][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.34%
[ 2018-05-15 00:39:35,243][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estim

In [11]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-05-15 00:40:11,171][cascade_classifier.transform] X_groups_test.shape=[(16500, 27)]
[ 2018-05-15 00:40:11,173][cascade_classifier.transform] group_dims=[27]
[ 2018-05-15 00:40:11,175][cascade_classifier.transform] X_test.shape=(16500, 27)
[ 2018-05-15 00:40:11,177][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(16500, 27)


Test Accuracy of ExtraTrees = 97.042424 %


In [12]:
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[15968     2]
 [  486    44]]


             precision    recall  f1-score   support

        0.0       0.97      1.00      0.98     15970
        1.0       0.96      0.08      0.15       530

avg / total       0.97      0.97      0.96     16500



In [13]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-05-15 00:40:35,688][cascade_classifier.transform] X_groups_test.shape=[(16500, 27)]
[ 2018-05-15 00:40:35,691][cascade_classifier.transform] group_dims=[27]
[ 2018-05-15 00:40:35,692][cascade_classifier.transform] X_test.shape=(16500, 27)
[ 2018-05-15 00:40:35,694][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(16500, 27)


In [14]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-05-15 00:40:38,230][cascade_classifier.fit_transform] X_groups_train.shape=[(33500, 29)],y_train.shape=(33500,),X_groups_test.shape=[(16500, 29)],y_test.shape=(16500,)
[ 2018-05-15 00:40:38,234][cascade_classifier.fit_transform] group_dims=[29]
[ 2018-05-15 00:40:38,236][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 00:40:38,238][cascade_classifier.fit_transform] group_ends=[29]
[ 2018-05-15 00:40:38,240][cascade_classifier.fit_transform] X_train.shape=(33500, 29),X_test.shape=(16500, 29)
[ 2018-05-15 00:40:38,244][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(33500, 29), X_cur_test.shape=(16500, 29)
[ 2018-05-15 00:40:41,234][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.85%
[ 2018-05-15 00:40:44,986][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.82%
[ 2018-05-15 00:40:48,690][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estim

In [16]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-05-15 00:41:18,977][cascade_classifier.transform] X_groups_test.shape=[(16500, 29)]
[ 2018-05-15 00:41:18,980][cascade_classifier.transform] group_dims=[29]
[ 2018-05-15 00:41:18,981][cascade_classifier.transform] X_test.shape=(16500, 29)
[ 2018-05-15 00:41:18,983][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(16500, 29)


Test Accuracy of ExtraTrees = 97.775758 %


In [17]:
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[15940    30]
 [  337   193]]


             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99     15970
        1.0       0.87      0.36      0.51       530

avg / total       0.98      0.98      0.97     16500



In [18]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')
    
    # Matrix de confusion
    from sklearn.metrics import classification_report, confusion_matrix
    print(confusion_matrix(y_test,y_pred))
    print('\n')
    print(classification_report(y_test,y_pred))

In [19]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-05-15 00:42:43,895][cascade_classifier.fit_transform] X_groups_train.shape=[(33500, 29)],y_train.shape=(33500,),X_groups_test.shape=[(16500, 29)],y_test.shape=(16500,)
[ 2018-05-15 00:42:43,899][cascade_classifier.fit_transform] group_dims=[29]
[ 2018-05-15 00:42:43,900][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 00:42:43,901][cascade_classifier.fit_transform] group_ends=[29]
[ 2018-05-15 00:42:43,902][cascade_classifier.fit_transform] X_train.shape=(33500, 29),X_test.shape=(16500, 29)
[ 2018-05-15 00:42:43,909][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(33500, 29), X_cur_test.shape=(16500, 29)
[ 2018-05-15 00:42:47,646][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.67%
[ 2018-05-15 00:42:50,875][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.70%
[ 2018-05-15 00:42:54,250][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estim

('Test Accuracy of ExtraTrees = 97.715152 %', 0, 'fois')
[[15945    25]
 [  352   178]]


             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99     15970
        1.0       0.88      0.34      0.49       530

avg / total       0.98      0.98      0.97     16500



[ 2018-05-15 00:43:27,427][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.67%
[ 2018-05-15 00:43:31,079][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.00%
[ 2018-05-15 00:43:35,214][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.64%
[ 2018-05-15 00:43:39,673][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.73%
[ 2018-05-15 00:43:43,675][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.73%
[ 2018-05-15 00:43:47,449][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.58%
[ 2018-05-15 00:43:50,796][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.79%
[ 2018-05-15 00:43:54,450][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.79%


('Test Accuracy of ExtraTrees = 97.606061 %', 1, 'fois')
[[15946    24]
 [  371   159]]


             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99     15970
        1.0       0.87      0.30      0.45       530

avg / total       0.97      0.98      0.97     16500



[ 2018-05-15 00:44:08,520][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.58%
[ 2018-05-15 00:44:12,061][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.76%
[ 2018-05-15 00:44:15,376][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.76%
[ 2018-05-15 00:44:19,157][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.82%
[ 2018-05-15 00:44:22,406][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.82%
[ 2018-05-15 00:44:27,399][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.46%
[ 2018-05-15 00:44:31,497][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.67%
[ 2018-05-15 00:44:35,539][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.97%


...

('Test Accuracy of ExtraTrees = 97.230303 %', 17, 'fois')
[[15952    18]
 [  439    91]]


             precision    recall  f1-score   support

        0.0       0.97      1.00      0.99     15970
        1.0       0.83      0.17      0.28       530

avg / total       0.97      0.97      0.96     16500



[ 2018-05-15 00:53:42,887][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.43%
[ 2018-05-15 00:53:46,127][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.46%
[ 2018-05-15 00:53:49,367][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.34%
[ 2018-05-15 00:53:52,499][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.61%
[ 2018-05-15 00:53:55,893][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.58%
[ 2018-05-15 00:53:59,575][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.34%
[ 2018-05-15 00:54:02,948][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.34%
[ 2018-05-15 00:54:05,964][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.61%


('Test Accuracy of ExtraTrees = 97.260606 %', 18, 'fois')
[[15957    13]
 [  439    91]]


             precision    recall  f1-score   support

        0.0       0.97      1.00      0.99     15970
        1.0       0.88      0.17      0.29       530

avg / total       0.97      0.97      0.96     16500



[ 2018-05-15 00:54:17,840][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.34%
[ 2018-05-15 00:54:21,007][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.34%
[ 2018-05-15 00:54:24,429][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.37%
[ 2018-05-15 00:54:28,015][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.37%
[ 2018-05-15 00:54:31,348][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.46%
[ 2018-05-15 00:54:34,606][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.52%
[ 2018-05-15 00:54:37,706][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.28%
[ 2018-05-15 00:54:40,849][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.58%


('Test Accuracy of ExtraTrees = 97.206061 %', 19, 'fois')
[[15954    16]
 [  445    85]]


             precision    recall  f1-score   support

        0.0       0.97      1.00      0.99     15970
        1.0       0.84      0.16      0.27       530

avg / total       0.97      0.97      0.96     16500



# ('Test Accuracy of ExtraTrees = 97.715152 %', 0, 'fois')